# Dynamics

In the previous tutorials we created Atoms objects and used Calculators to get properties including energies and forces... but we didn't do very much with this information.

A very useful thing to do with this information is to update our strucuture! ASE includes algorithms for molecular dynamics, geometry optimization and global optimization. It is a useful toolkit for experimentation and method-development in this area, or development of multi-step pipelines.

## Molecular dynamics

Molecular dynamics (MD) methods are found in [ase.md](https://wiki.fysik.dtu.dk/ase/ase/md.html).

> This is not a course in MD; be aware that thermostats and timesteps should be chosen with care for a given research problem!

Starting with the EMT potential, let us try some simulated annealing of a cubic Cu nanoparticle. First we prepare the structure and assign some initial momenta.

In [1]:
import ase.build
from ase.calculators.emt import EMT
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution

# Set up a crystal
cu_cube = ase.build.bulk('Cu', cubic=True) * [3, 3, 3]

# Describe the interatomic interactions with the Effective Medium Theory
cu_cube.calc = EMT()

# Set the momenta corresponding to T=300K
MaxwellBoltzmannDistribution(cu_cube, temperature_K=300)

There are stores on the atoms as velocities. The initial forces should be very small due to symmetry.

In [2]:
row_limit = 4
print('velocities:')
print(cu_cube.get_velocities()[:row_limit])

print('\nforces:')
print(cu_cube.get_forces()[:row_limit])

print(f'\nkinetic energy: {cu_cube.get_kinetic_energy()}')

velocities:
[[-2.40486831e-02 -2.16358278e-02  1.99401934e-03]
 [ 1.12755483e-02  3.93652296e-03 -2.57274669e-02]
 [-8.04195899e-04 -1.42356161e-02 -6.98740581e-03]
 [ 1.96323098e-02  6.48999019e-03 -5.42692077e-05]]

forces:
[[ 1.05748743e-14  9.68843061e-15  8.49667559e-15]
 [ 2.47839943e-14 -1.35655376e-15 -2.27769192e-15]
 [-1.81799020e-15  1.80896964e-14 -1.32012457e-15]
 [-2.18835366e-15  1.32706346e-16  1.38292156e-14]]

kinetic energy: 3.7843533764007526


Next the dynamics object is created and the Atoms are attached to it. For constant-energy MD we can use the Velocity Verlet method.

While the default distance and energy units (Angstrom and eV) in ASE are fairly friendly, the related time unit is a bit awkward so we use a unit conversion from `ase.units` to set a timestep of 5 fs.

In [3]:
from ase import units
from ase.md.verlet import VelocityVerlet

dyn = VelocityVerlet(cu_cube, 5 * units.fs)

dyn.run(1)


print('velocities:')
print(cu_cube.get_velocities()[:row_limit])

print('\nforces:')
print(cu_cube.get_forces()[:row_limit])

velocities:
[[-0.02352488 -0.02096951  0.00200228]
 [ 0.01113474  0.00390456 -0.02531237]
 [-0.00048695 -0.01406838 -0.00700438]
 [ 0.01940485  0.00638033 -0.0002652 ]]

forces:
[[ 0.13554452  0.17242442  0.00213891]
 [-0.03643847 -0.008272    0.10741521]
 [ 0.08209344  0.04327661 -0.00439152]
 [-0.05886154 -0.0283771  -0.054584  ]]


After one timestep, we see that the velocities have changes slightly and significant forces have appeared.
Running some more time steps, while occasionally printing the kinetic and potential energy:

In [4]:
from ase import Atoms
def printenergy(atoms: Atoms) -> None:
    """Function to print the potential, kinetic and total energy"""
    epot = atoms.get_potential_energy() / len(atoms)
    ekin = atoms.get_kinetic_energy() / len(atoms)
    temperature = ekin / (1.5 * units.kB)
    
    print(f'Energy per atom: Epot = {epot:.3f}eV  Ekin = {ekin:.3f}eV '
          f'(T={temperature:3.0f}K)  Etot = {epot+ekin:.3f}eV')

# Now run the dynamics
printenergy(cu_cube)
for i in range(20):
    dyn.run(10)
    printenergy(cu_cube)

Energy per atom: Epot = -0.005eV  Ekin = 0.034eV (T=263K)  Etot = 0.029eV
Energy per atom: Epot = 0.019eV  Ekin = 0.011eV (T= 85K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.017eV (T=128K)  Etot = 0.030eV
Energy per atom: Epot = 0.012eV  Ekin = 0.017eV (T=133K)  Etot = 0.029eV
Energy per atom: Epot = 0.009eV  Ekin = 0.020eV (T=157K)  Etot = 0.029eV
Energy per atom: Epot = 0.011eV  Ekin = 0.018eV (T=140K)  Etot = 0.029eV
Energy per atom: Epot = 0.015eV  Ekin = 0.014eV (T=109K)  Etot = 0.029eV
Energy per atom: Epot = 0.008eV  Ekin = 0.021eV (T=163K)  Etot = 0.029eV
Energy per atom: Epot = 0.014eV  Ekin = 0.016eV (T=121K)  Etot = 0.030eV
Energy per atom: Epot = 0.010eV  Ekin = 0.019eV (T=148K)  Etot = 0.029eV
Energy per atom: Epot = 0.012eV  Ekin = 0.018eV (T=138K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.016eV (T=126K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.017eV (T=130K)  Etot = 0.029eV
Energy per atom: Epot = 0.006eV  Ekin = 0.023eV (T

As the system equilibrates, energy is conserved but the temperature is about half of the original setting. *Why?*

It's a bit inconvenient to write a loop that stops and starts the MD as above; instead we can attach an "observer" function.

In [5]:
# Run a few more steps
def energy_observer():
    printenergy(cu_cube)
    
dyn.attach(energy_observer, interval=10)
dyn.run(100)

Energy per atom: Epot = 0.012eV  Ekin = 0.018eV (T=138K)  Etot = 0.029eV
Energy per atom: Epot = 0.011eV  Ekin = 0.019eV (T=145K)  Etot = 0.029eV
Energy per atom: Epot = 0.012eV  Ekin = 0.017eV (T=133K)  Etot = 0.029eV
Energy per atom: Epot = 0.012eV  Ekin = 0.018eV (T=138K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.016eV (T=127K)  Etot = 0.029eV
Energy per atom: Epot = 0.010eV  Ekin = 0.019eV (T=150K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.016eV (T=125K)  Etot = 0.030eV
Energy per atom: Epot = 0.011eV  Ekin = 0.018eV (T=141K)  Etot = 0.029eV
Energy per atom: Epot = 0.013eV  Ekin = 0.016eV (T=127K)  Etot = 0.029eV
Energy per atom: Epot = 0.011eV  Ekin = 0.019eV (T=144K)  Etot = 0.029eV


True

**Exercise:** use an observer to measure the average (root mean-square) displacement of Cu atoms at 300K

In [6]:
import nglview

def show(atoms: ase.Atoms) -> None:
    view = nglview.show_ase(atoms)
    if any(atoms.pbc):
        view.add_unitcell()
    return view

show(cu_cube)

NGLWidget()

At modest temperature and under periodic boundary conditions, the atoms have not moved very far. Let's try something more extreme: remove the boundary conditions, ramp up the temperature, and use a Langevin thermostat that regulates the temperature. This will take a few minutes to run.

In [10]:
Langevin?

Init signature:
Langevin(
    atoms,
    timestep,
    temperature=None,
    friction=None,
    fixcm=True,
    *,
    temperature_K=None,
    trajectory=None,
    logfile=None,
    loginterval=1,
    communicator=<ase.parallel.MPI object at 0x7f712908f3a0>,
    rng=None,
    append_trajectory=False,
)
Docstring:      Langevin (constant N, V, T) molecular dynamics.
Init docstring:
Parameters:

atoms: Atoms object
    The list of atoms.

timestep: float
    The time step in ASE time units.

temperature: float (deprecated)
    The desired temperature, in electron volt.

temperature_K: float
    The desired temperature, in Kelvin.

friction: float
    A friction coefficient, typically 1e-4 to 1e-2.

fixcm: bool (optional)
    If True, the position and momentum of the center of mass is
    kept unperturbed.  Default: True.

rng: RNG object (optional)
    Random number generator, by default numpy.random.  Must have a
    standard_normal method matching the signature of
    numpy.random.stan

In [19]:
from ase.io.trajectory import Trajectory
from ase.md import Langevin

cu_lump = cu_cube.copy()
cu_lump.pbc=False
cu_lump.calc = EMT()

def energy_observer():
    printenergy(cu_lump)

dyn = Langevin(cu_lump, 5 * units.fs, friction=0.005, temperature_K=1000)
dyn.attach(energy_observer, interval=50)

# We also want to save the positions of all atoms after every 50th time step.
traj = Trajectory('cu_melt.traj', 'w', cu_lump)
dyn.attach(traj.write, interval=50)

# Now run the dynamics
dyn.run(1000)


Energy per atom: Epot = 0.453eV  Ekin = 0.018eV (T=143K)  Etot = 0.472eV
Energy per atom: Epot = 0.457eV  Ekin = 0.038eV (T=293K)  Etot = 0.495eV
Energy per atom: Epot = 0.470eV  Ekin = 0.048eV (T=372K)  Etot = 0.518eV
Energy per atom: Epot = 0.488eV  Ekin = 0.055eV (T=426K)  Etot = 0.543eV
Energy per atom: Epot = 0.497eV  Ekin = 0.065eV (T=502K)  Etot = 0.562eV
Energy per atom: Epot = 0.501eV  Ekin = 0.078eV (T=607K)  Etot = 0.580eV
Energy per atom: Epot = 0.513eV  Ekin = 0.074eV (T=572K)  Etot = 0.587eV
Energy per atom: Epot = 0.501eV  Ekin = 0.093eV (T=723K)  Etot = 0.595eV
Energy per atom: Epot = 0.520eV  Ekin = 0.089eV (T=687K)  Etot = 0.608eV
Energy per atom: Epot = 0.522eV  Ekin = 0.094eV (T=729K)  Etot = 0.616eV
Energy per atom: Epot = 0.521eV  Ekin = 0.100eV (T=771K)  Etot = 0.621eV
Energy per atom: Epot = 0.521eV  Ekin = 0.104eV (T=808K)  Etot = 0.626eV
Energy per atom: Epot = 0.542eV  Ekin = 0.086eV (T=663K)  Etot = 0.628eV
Energy per atom: Epot = 0.541eV  Ekin = 0.090eV (T=

True

The trajectory file lets us visualise what happened.

In [20]:
from ase.visualize import view
import ase.io

frames = ase.io.read('cu_melt.traj', index=':')
view(frames)

<Popen: returncode: None args: ['/home/adam/micromamba/envs/ase-tutorials/bi...>

X Error of failed request:  BadValue (integer parameter out of range for operation)
  Major opcode of failed request:  12 (X_ConfigureWindow)
  Value in failed request:  0x0
  Serial number of failed request:  368
  Current serial number in output stream:  368


Having reached temperature we can also steadily quench by reducing the target temperature over the course of the simulation.

In [21]:
for temperature in range(800, 100, -50):
    dyn.set_temperature(temperature_K=temperature)
    dyn.run(100)

Energy per atom: Epot = 0.555eV  Ekin = 0.109eV (T=840K)  Etot = 0.664eV
Energy per atom: Epot = 0.546eV  Ekin = 0.119eV (T=924K)  Etot = 0.665eV
Energy per atom: Epot = 0.550eV  Ekin = 0.109eV (T=843K)  Etot = 0.659eV
Energy per atom: Epot = 0.533eV  Ekin = 0.118eV (T=915K)  Etot = 0.651eV
Energy per atom: Epot = 0.540eV  Ekin = 0.104eV (T=805K)  Etot = 0.644eV
Energy per atom: Epot = 0.546eV  Ekin = 0.103eV (T=794K)  Etot = 0.649eV
Energy per atom: Epot = 0.529eV  Ekin = 0.110eV (T=850K)  Etot = 0.639eV
Energy per atom: Epot = 0.526eV  Ekin = 0.107eV (T=831K)  Etot = 0.634eV
Energy per atom: Epot = 0.538eV  Ekin = 0.088eV (T=682K)  Etot = 0.626eV
Energy per atom: Epot = 0.525eV  Ekin = 0.094eV (T=728K)  Etot = 0.619eV
Energy per atom: Epot = 0.526eV  Ekin = 0.088eV (T=678K)  Etot = 0.614eV
Energy per atom: Epot = 0.517eV  Ekin = 0.094eV (T=729K)  Etot = 0.611eV
Energy per atom: Epot = 0.510eV  Ekin = 0.098eV (T=760K)  Etot = 0.608eV
Energy per atom: Epot = 0.515eV  Ekin = 0.091eV (T=

In [22]:
frames = ase.io.read('cu_melt.traj', index=':')
view(frames)

<Popen: returncode: None args: ['/home/adam/micromamba/envs/ase-tutorials/bi...>

X Error of failed request:  BadValue (integer parameter out of range for operation)
  Major opcode of failed request:  12 (X_ConfigureWindow)
  Value in failed request:  0x0
  Serial number of failed request:  368
  Current serial number in output stream:  368


We see that as the temperature falls some order returns to the structure.

## Silicon

In [ ]:
from pathlib import Path
import quippy.potential_module
from quippy.potential import Potential

gap = Potential(param_filename=str(Path.cwd() / 'Si_PRX_GAP/gp_iter6_sparse9k.xml'))

In [15]:
from ase import Atoms
def printenergy(atoms: Atoms) -> None:
    """Function to print the potential, kinetic and total energy"""
    epot = atoms.get_potential_energy() / len(atoms)
    ekin = atoms.get_kinetic_energy() / len(atoms)
    temperature = ekin / (1.5 * ase.units.kB)
    
    print(f'Energy per atom: Epot = {epot:.3f}eV  Ekin = {ekin:.3f}eV '
          f'(T={temperature:3.0f}K)  Etot = {epot+ekin:.3f}eV')

In [ ]:
import ase.build
import ase.units

si = ase.build.bulk('Si', cubic=True) * [2, 2, 2]
si.calc = gap
si.calc.name = 'Si melt'

from ase.io.trajectory import Trajectory
from ase.md import Langevin

def energy_observer():
    printenergy(si)

dyn = Langevin(si, 5 * ase.units.fs, friction=0.005, temperature_K=1000)
dyn.attach(energy_observer, interval=50)

# We also want to save the positions of all atoms after every 50th time step.
traj = Trajectory('si_melt.traj', 'w', si)
dyn.attach(traj.write, interval=50)

# Now run the dynamics
dyn.run(1000)


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

Energy per atom: Epot = -163.176eV  Ekin = 0.000eV (T=  0K)  Etot = -163.176eV


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

Energy per atom: Epot = -163.164eV  Ekin = 0.014eV (T=107K)  Etot = -163.150eV


pplication may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Ple

...

OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec

Energy per atom: Epot = -163.143eV  Ekin = 0.038eV (T=294K)  Etot = -163.105eV


with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option

Energy per atom: Epot = -163.134eV  Ekin = 0.052eV (T=400K)  Etot = -163.083eV


OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detect OpenMP Loop and this application may hang. Please rebuild the library with USE_OPENMP=1 option.
OpenBLAS Warning : Detec